https://note.nkmk.me/en/python-re-match-object-span-group/

In [115]:
lines = [
"G1 F1800",
"G1 E-6.5 F2400",
"G92 E0",
"M82",
"G1 X119.988 Y87.177 F1800 ;this is a comment",
"G1 E6.5 F2400",
"M203 X10",
"M20",
";TYPE:Skirt/Brim",
"No command"]

In [50]:
import re

In [82]:
commands = {
        "G0 ": None,  # non Extrusion Move
        "G1 ": None,  # Extrusion Move
        "G4 ": None,  # Dwell
        "M82": None,  # E Absolute
        "M203 ": None,  # Max Feedrate
        "M204 ": None,  # Starting Acceleration
        "M205 ": None,  # Advanced Settings
        "M83": None,  # E Relative
        "G20": None,  # Inches
        "G21": None,  # Milimeters
        "G90": None,  # Absolute Positioning
        "G91": None,  # Relative Positioning
        "G92 ": None, # Set Position
        ";": None  # Comment
    }

In [121]:
for line in lines:
    line_dict = dict()
    matches = list()
    for command in commands.keys():
        match = re.search(command,line)
        if match is not None:
            matches.append(match)
    # check for longest match? more robust 
    
    # support for multiple commands or additional comments per line
    for i in range(len(matches)):
        match_0 = matches[i]
        match_1 = matches[i+1].start() if i+1 < len(matches) else len(line)
        command_type = match_0.group()
        command_content = line[match_0.end():match_1]
        line_dict[command_type] = command_content
    print(line_dict)

{'G1 ': 'F1800'}
{'G1 ': 'E-6.5 F2400'}
{'G92 ': 'E0'}
{'M82': ''}
{'G1 ': 'X119.988 Y87.177 F1800 ', ';': 'this is a comment'}
{'G1 ': 'E6.5 F2400'}
{'M203 ': 'X10'}
{}
{';': 'TYPE:Skirt/Brim'}
{}


In [78]:
def get_command(line):
    command_begin = -1
    for command in commands.keys():
        tmp = line.find(command)
        if command_begin < tmp:
            command_begin = tmp 

    print(command_begin,command) if command_begin != -1 else None

In [19]:
def line_to_dict(line):
    """
    Converts known GCode commands to dictionary.
    """
    commands = {
        "G0": None,  # non Extrusion Move
        "G1": None,  # Extrusion Move
        "G4": None,  # Dwell
        "M203": None,  # Max Feedrate
        "M204": None,  # Starting Acceleration
        "M205": None,  # Advanced Settings
        "M82": None,  # E Absolute
        "M83": None,  # E Relative
        "G20": None,  # Inches
        "G21": None,  # Milimeters
        "G90": None,  # Absolute Positioning
        "G91": None,  # Relative Positioning
        ";": None,  # comment
    }

In [47]:
keys = tuple(commands.keys())

In [48]:
type(keys)

tuple

In [75]:
for line in lines:
    get_command(line)


0 G1 
0 G92 
3 G1 
0 M82
0 G1 
0 M203 
0 ;
